# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 16 2022 1:03PM,246634,12,HH-36876,Hush Hush,Released
1,Sep 16 2022 1:03PM,246634,12,HH-36953,Hush Hush,Released
2,Sep 16 2022 1:03PM,246634,12,HH-36954,Hush Hush,Released
3,Sep 16 2022 1:03PM,246634,12,HH-36955,Hush Hush,Released
4,Sep 16 2022 1:03PM,246634,12,HH-36956,Hush Hush,Released
5,Sep 16 2022 1:03PM,246634,12,HH-36957,Hush Hush,Released
6,Sep 16 2022 1:03PM,246634,12,HH-36958,Hush Hush,Released
7,Sep 16 2022 1:01PM,246633,10,SONSB0001941,"Nextsource Biotechnology, LLC",Released
8,Sep 16 2022 1:00PM,246632,20,ALU0007940,Alumier Labs Inc.,Released
9,Sep 16 2022 1:00PM,246632,20,ALU0007942,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,246630,Released,3
18,246631,Released,2
19,246632,Released,2
20,246633,Released,1
21,246634,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246630,NaN,3.0
246631,NaN,2.0
246632,NaN,2.0
246633,NaN,1.0
246634,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246578,0.0,2.0
246595,0.0,1.0
246597,0.0,1.0
246598,1.0,9.0
246607,5.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246578,0,2
246595,0,1
246597,0,1
246598,1,9
246607,5,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246578,0,2
1,246595,0,1
2,246597,0,1
3,246598,1,9
4,246607,5,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246578,,2
1,246595,,1
2,246597,,1
3,246598,1,9
4,246607,5,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 16 2022 1:03PM,246634,12,Hush Hush
7,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC"
8,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.
10,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.
12,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.
15,Sep 16 2022 12:43PM,246627,21,"NBTY Global, Inc."
16,Sep 16 2022 12:42PM,246628,21,"NBTY Global, Inc."
17,Sep 16 2022 12:41PM,246629,22,FUJIFILM Diosynth Biotechnologies Texas LLC
18,Sep 16 2022 11:54AM,246626,21,"NBTY Global, Inc."
19,Sep 16 2022 11:46AM,246621,15,"Carwin Pharmaceutical Associates, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 16 2022 1:03PM,246634,12,Hush Hush,,7
1,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC",,1
2,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,,2
3,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,,2
4,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,,3
5,Sep 16 2022 12:43PM,246627,21,"NBTY Global, Inc.",,1
6,Sep 16 2022 12:42PM,246628,21,"NBTY Global, Inc.",,1
7,Sep 16 2022 12:41PM,246629,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
8,Sep 16 2022 11:54AM,246626,21,"NBTY Global, Inc.",1,
9,Sep 16 2022 11:46AM,246621,15,"Carwin Pharmaceutical Associates, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 1:03PM,246634,12,Hush Hush,7,
1,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC",1,
2,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,2,
3,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,2,
4,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,3,
5,Sep 16 2022 12:43PM,246627,21,"NBTY Global, Inc.",1,
6,Sep 16 2022 12:42PM,246628,21,"NBTY Global, Inc.",1,
7,Sep 16 2022 12:41PM,246629,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
8,Sep 16 2022 11:54AM,246626,21,"NBTY Global, Inc.",,1
9,Sep 16 2022 11:46AM,246621,15,"Carwin Pharmaceutical Associates, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 1:03PM,246634,12,Hush Hush,7,
1,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC",1,
2,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,2,
3,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,2,
4,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 1:03PM,246634,12,Hush Hush,7.0,NaN
1,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC",1.0,NaN
2,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,2.0,NaN
3,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,2.0,NaN
4,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 1:03PM,246634,12,Hush Hush,7.0,0.0
1,Sep 16 2022 1:01PM,246633,10,"Nextsource Biotechnology, LLC",1.0,0.0
2,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,2.0,0.0
3,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,2.0,0.0
4,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,986456,32.0,6.0
12,246634,7.0,0.0
15,493240,9.0,12.0
16,739770,4.0,0.0
20,1233118,8.0,2.0
21,739881,2.0,1.0
22,246629,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,986456,32.0,6.0
1,12,246634,7.0,0.0
2,15,493240,9.0,12.0
3,16,739770,4.0,0.0
4,20,1233118,8.0,2.0
5,21,739881,2.0,1.0
6,22,246629,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,32.0,6.0
1,12,7.0,0.0
2,15,9.0,12.0
3,16,4.0,0.0
4,20,8.0,2.0
5,21,2.0,1.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,12,Released,7.0
2,15,Released,9.0
3,16,Released,4.0
4,20,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,21,22
Status,,,,,,,
Executing,6.0,0.0,12.0,0.0,2.0,1.0,0.0
Released,32.0,7.0,9.0,4.0,8.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,21,22
0,Executing,6.0,0.0,12.0,0.0,2.0,1.0,0.0
1,Released,32.0,7.0,9.0,4.0,8.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,21,22
0,Executing,6.0,0.0,12.0,0.0,2.0,1.0,0.0
1,Released,32.0,7.0,9.0,4.0,8.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()